# **Spotify Web API Task 3 : Obtaining an Artists Top Tracks**

### **Background** ###
This notebook provides a step by step guide to obtaining a given artists top tracks on Spotify. We will access the Spotify Web API in order to do so. The program will ask the user to input the name of an artist and will return a list of their top ten tracks

### **Prerequisites**
Before proceeding, you need to set up a Spotify Developer App and obtain the following details:

- **Client ID**: Your application's unique identifier.

- **Client Secret** : A secret key for your application, used to authenticate your requests.
<br>

These credentials can be found on your app's dashboard on the Spotify Developer Portal. Make sure to never hardcode your client ID and client secret directly in your scripts due to security risks. Instead, store them securely, such as in an environment file (.env), and load them into your program as needed.

### **Step by Step Guide**

Here is a step by step guide to the process : 

1) **Import Libraries :** Importing the relevant libraries.

2) **Import Secrets :** Initialise the .env file and retrieve sensitive information from within.

3) **Make Function to Obtain Access Token :** Gain access token from the Spotify Web API 

4) **Make Function to Make HTTP Headers :** Make headers to use during the Authorisation process.

5) **Make Function Which Searches for the Artist :** This will gather information on the specified artist and retrieve their name and artist id.

6) **Make Function to Search for the Artists Songs :** This will retrieve the specified artists top tracks on Spotify.

7) **Make a Composite Function :** This will combine all functions into one function which can be called all at once. 

8) **Test Program :** This will test the program and obtain the band SAULT's top tracks.

In [1]:
# Imports 
from dotenv import load_dotenv
import os 
import base64
from requests import post, get
import json

In [2]:
# This function gets the authorisation token for our app from the Spotify Web API. 
# In order to do, we need to use the client_id, client_secret and encode it using the base64 library.
def get_token() : 
    # Initialise the secrets within .env file 
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    # Concatenate our id and secret in order to create the authorisation string 
    auth_string = client_id + ":" + client_secret
    # Convert the authorisation string into bytes using UTF-8 encoding
    auth_bytes = auth_string.encode("utf-8")
    # Take the byte sequence and encode it with base 64 encoding
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    # Making a request to the Spotify Web API
    # This specifes the URL of the Spotify API endpoint which issues access tokens 
    url = 'https://accounts.spotify.com/api/token'
    # This contains the HTTP headers that are required to authenicate and specify the requests content type 
    # Authorization is the encoded authorisation string 
    # Content-Type specifies the format of the data being sent to the API  
    headers = {
        "Authorization" : "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    # Grant_type is used to obtain an access token without requiring user interaction, used typically for server-server connections
    data = {"grant_type": "client_credentials"}
    # Stores the response from the HTTP post request made to the Spotify API, specifying the url, headers and grant type we defined earlier 
    result = post(url, headers = headers, data = data)
    # Parsing the JSON response from the API
    json_result = json.loads(result.content)
    # Extracting the access token 
    token = json_result["access_token"]
    return token 

In [3]:
# Function to get the correct authorisation header for the API endpoint 
def get_auth_header (token) : 
    return {"Authorization": "Bearer " + token}

In [4]:
# Function which locates the artists name and id given the user input
def search_for_artist(token) : 

    # Ask the user to input the artist name 
    artist_search = input('What Artist do you want to search for?')
    # base url for Spotifys URL endpoint 
    url = "https://api.spotify.com/v1/search"
    # Call the header function to create the authorization header from earlier 
    headers = get_auth_header(token)
    # Prepare the query parameters, use the API documentation to understand this better
    # Limited to the first response
    query = f'q={artist_search}&type=artist&limit=1'
    query_url = url + '?' + query 
    # Send the GET request to get the Spotify API 
    result = get(query_url, headers = headers)
    # Parse through the JSON response 
    json_result = json.loads(result.content)['artists']['items']
    # Check if any artist was found given the user inputted artist search term 
    if len(json_result) == 0: 
        print('No Artist with this name')
        return None 
    # Return the first artist result
    result = json_result[0]
    artist_info = []
    artist_name = result['name']
    artist_id = result['id']
    artist_info.append(artist_name)
    artist_info.append(artist_id)
    return artist_info

In [12]:
# Function which gets the top tracks, given a token and list of both the artists name and id 
def get_top_tracks() : 
    token = get_token()
    artist_info = search_for_artist(token)

    artist_name = artist_info[0]
    artist_id = artist_info[1]

     # Construct the URL for the Spotify Web API endpoint 
    url = f'https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US'
     # Generate the headers using the function we made earlier 
    headers = get_auth_header(token)
     # Send the GET request
    result = get(url, headers = headers)
     # Parse through the JSON response 
    json_result = json.loads(result.content)['tracks']
    songs = json_result
    
    # Formatting the output of the function
    print(f"{artist_name}'s Top Tracks are :")
    print()
    for idx, song in enumerate(songs) : 
          print(f"{idx + 1}. {song['name']}")

In [13]:
get_top_tracks()

SAULT's Top Tracks are :

1. Wildfires
2. Why Why Why Why Why
3. Envious
4. Masterpiece
5. Let Me Go
6. Fight for Love
7. Son Shine
8. Free
9. Colour Blind
10. Up All Night
